In [1]:
import os
import torch
from scipy.stats import special_ortho_group
from toil.job import Job
from molmimic.common.voxels import ProteinVoxelizer
from molmimic.common.featurizer import ProteinFeaturizer

In [7]:
def get_structure(pdb_file):
    """Read structure and features (or calculate if not present)"""
    features_path = os.path.dirname(pdb_file)
    try:
        protein = ProteinVoxelizer(
            pdb_file, 
            os.path.splitext(os.path.basename(pdb_file))[0], 
            features_path=features_path)
    except FileNotFoundError:
        print("---")
        #Calculate features
        protein = ProteinFeaturizer(
            pdb_file, 
            os.path.splitext(os.path.basename(pdb_file))[0], 
            Job(), 
            os.path.dirname(pdb_file), 
            force_feature_calculation=True)
        
        #Only calculate accesible surface area
        [protein.get_accessible_surface_area_residue(protein._remove_altloc(a)) for a in protein.get_atoms()]
        
        #Try reading again
        protein = ProteinVoxelizer(
            pdb_file, 
            os.path.splitext(os.path.basename(pdb_file))[0], 
            features_path=features_path)
    
    return protein

In [3]:
def dock_pair(receptor_file, ligand_file, rotations=100):
    #Read receptor and features (or calculate if not present)
    receptor = get_structure(receptor_file)
        
    #Read ligand and features (or calculate if not present)
    ligand = get_structure(ligand_file)
        
    #Map receptor to voxels
    receptor_indices, receptor_data, _ = receptor.map_atoms_to_voxel_space(
        autoencoder=True,
        only_surface=False,
        simple_fft=True)
    receptor_volume = torch.zeros(256,256,256)
    receptor_volume[receptor_indices] = receptor_data
    del receptor_indices, receptor_data, _
    receptor_volume.cuda()
    
    #Create Fourier image of receptor
    receptor_fft = torch.fft(receptor_volume)
    
    del receptor_volume
    
    best_ligand = None
    best_energy = None
    
    for i in range(rotations):
        rot_mat = special_ortho_group.rvs(3)
        print("Running rotation i:", rot_mat)
        
        #Rotate ligand
        next(ligand.rotate(rot_mat))
        
        #Map ligand to voxels
        ligand_indices, ligand_data, _ = ligand.map_atoms_to_voxel_space(
            autoencoder=True,
            only_surface=False,
            simple_fft=True)
        ligand_volume = torch.zeros(256,256,256)
        ligand_volume[ligand_indices] = ligand_data
        del ligand_indices, ligand_data, _
        ligand_volume.cuda()
        
        #Create Fourier image of rotated ligand
        ligand_fft = torch.fft(ligand_volume)
        
        #Calculate energy using the convolution thm
        energy = torch.ifft(receptor_fft*ligand_fft)
        
        del ligand_volume, ligand_fft
        
        if energy<best_energy:
            best_energy = energy.item()
            best_ligand = ligand
            del energy
            return
        
        del ligand
    
    return best_ligand, best_energy

In [8]:
ligand, energy = dock_pair("BM5-clean/HADDOCK-ready/1A2K/1A2K_r_u.pdb", "BM5-clean/HADDOCK-ready/1A2K/1A2K_l_u.pdb")

atom_feature_mode r
---
force_feature_calculation w+
atom_feature_mode w+
atom_feature_mode r


FileNotFoundError: File BM5-clean/HADDOCK-ready/1A2K/1A2K_r_u_atom.h5 does not exist